In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
from nltk.tokenize import sent_tokenize
full = []
for dirname, _, filenames in os.walk('Train_Full'):
    for filename in filenames:
        with open(os.path.join("", os.path.join(dirname, filename)), 'r', encoding='UTF-16') as f:
            
            full.append(f.read())
        
            # raw_text = raw_text.replace("\n", " ")
            # raw_text = raw_text.lower()
            # # raw_text = sent_tokenize(raw_text)
            # for text in raw_text:
            #     full.append(text)
            
            f.close()

In [3]:
from underthesea import word_tokenize

from tqdm import tqdm
import string
import re
def get_token(list_str):
    temp=[]
    for raw_text in tqdm(list_str):
        raw_text = raw_text.lower()
        raw_text = raw_text.replace("\n", "")
        raw_text = raw_text.replace("- ", "")
        raw_text = sent_tokenize(raw_text)
        sentence_list = [sentence.translate(str.maketrans('', '', string.punctuation+"“"+"”")) for sentence in raw_text]
        for item in sentence_list:
            
            tokens = word_tokenize(item)
            temp.append(tokens)
        
    return temp




# sents = [re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', data_str) for data_str in data_str]

data = get_token(full)

100%|██████████| 42744/42744 [41:56<00:00, 16.99it/s]  


In [4]:
print(data[10])

['song', 'thực tế', 'việc', 'áp dụng', 'chủ trương', 'giảm', 'tải', 'với', 'các', 'trường', 'trên', 'địa bàn', 'thủ đô', 'hà nội', 'hiện nay', 'vẫn', 'thể hiện', 'sự', 'lúng túng', 'từ', 'chính', 'cán bộ', 'quản lý', 'đến', 'giáo viên', 'đứng lớp', '…', 'trong', 'buổi', 'tổng kết', '3', 'năm', 'thay', 'sách', 'ở', 'bậc', 'tiểu học', 'nhiều', 'ý kiến', 'cho', 'rằng', 'nhiều', 'bài', 'trong', 'chương trình', 'nặng', 'so', 'với', 'khả năng', 'tiếp thu', 'của', 'học sinh', 'hs']


In [5]:
from nltk.lm.preprocessing import padded_everygram_pipeline

train_data, padded_sents = padded_everygram_pipeline(3, data)

In [6]:
from nltk.lm import MLE, Laplace, KneserNeyInterpolated, WittenBellInterpolated
import pickle

n=3

vi_model = KneserNeyInterpolated(n)

vi_model.fit(train_data, padded_sents)
print(len(vi_model.vocab))

421355


In [10]:
import pickle
with open(os.path.join('kneserney_1st_ngram_model.pkl'), 'wb') as fout:
    pickle.dump(vi_model, fout)

In [41]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, pre_words=[]):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = pre_words
    for i in range(num_words):
        token = model.generate(1, text_seed=content[-2:])
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

generate_sent(vi_model, 10, ["tổ", "chức"])
#đất nước việt nam các phương tiện như ở vn sử dụng

'tổ chức vô địch mùa đông hà nội thời tiết ấm áp sẽ còn tăng đến'